In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext
import nengo_gui.ipython

## Flexible motor oscillators

Goal: Put in two different phonemes,
decode out two different functions.

In [ ]:
# Common to both implementations
from nengo.dists import Uniform

def zone(x):
    theta = np.arctan2(x[1], x[0])
    if theta > (7.0 / 8) * np.pi:
        return 0
    else:
        return x


def radial_f(fn):
    def _fn(x):
        #theta = math.arctan2(x[1], x[0])
        #t = theta / (2 * np.pi) + 0.5
        return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
    return _fn

In [ ]:
# Functions to decode from the oscillator
def f1(x):
     return 0.1 * np.sin(x * 2 * np.pi * 3) + (0.2 + x * 0.5)
def f2(x):
     return -0.2 * np.cos(x * 2 * np.pi * 5) - (0.2 + x * 0.5)
x = np.linspace(0, 1, 100)

plt.plot(x, f1(x))
plt.plot(x, f2(x))

First, let's show it working with separate oscillators.

In [ ]:
def temporal_f(fn, n_neurons=500, tau=0.025, freq=1.0, net=None):
    if net is None:
        net = nengo.Network(label='fn=%s' % fn.__name__)

    def _ensure_dict(dname):
        if not hasattr(net, dname):
            setattr(net, dname, {})

    _ensure_dict('osc_inputs')
    _ensure_dict('osc_outputs')
    _ensure_dict('oscillators')

    omega = tau * 2 * np.pi * freq
    encoders = [[np.cos(theta), np.sin(theta)]
                for theta in np.random.uniform(-np.pi, (7.0 / 8) * np.pi, n_neurons)]
    with net:
        net.osc_inputs[fn] = nengo.Node(output=None, size_in=1, label="%s in" % fn.__name__)
        net.osc_outputs[fn] = nengo.Node(output=None, size_in=1, label="%s out" % fn.__name__)
        net.oscillators[fn] = nengo.Ensemble(
            n_neurons, dimensions=2, intercepts=Uniform(0.3, 1), encoders=encoders, label=fn.__name__)
        nengo.Connection(net.osc_inputs[fn], net.oscillators[fn], transform=[[-1], [0]])
        nengo.Connection(net.oscillators[fn], net.oscillators[fn],
                         transform=[[1, -omega], [omega, 1]], function=zone, synapse=tau)
        nengo.Connection(net.oscillators[fn], net.osc_outputs[fn], function=radial_f(fn))
    return net

tau = 0.025
freq = 0.5
with nengo.Network(seed=1) as net:
    temporal_f(f1, net=net, tau=tau, freq=freq)
    temporal_f(f2, net=net, tau=tau, freq=freq)
    
    # We'll give each a little kick
    kick = nengo.Node(lambda t: .8 if 0.1 < t < 0.2 else 0.0)
    nengo.Connection(kick, net.osc_inputs[f1], synapse=None)
    nengo.Connection(kick, net.osc_inputs[f2], synapse=None)

In [ ]:
# nengo_gui.ipython.IPythonViz(net, cfg='cell1.cfg')

In [ ]:
with net:
    f1_p = nengo.Probe(net.osc_outputs[f1], synapse=0.01)
    f2_p = nengo.Probe(net.osc_outputs[f2], synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(1./freq + 0.2)

In [ ]:
x = np.arange(0, 1./freq, 0.001)
t = sim.trange()
plt.plot(t, sim.data[f1_p])
plt.plot(t, sim.data[f2_p])
plt.plot(x+0.1, f1(x*freq), c='k', lw=1)
plt.plot(x+0.1, f2(x*freq), c='k', lw=1)

comp_time = (t > 0.1) & (t < 0.1 + 1. / freq)
print("f1 rmse: %f" % npext.rmse(f1(x*freq), sim.data[f1_p][comp_time]))
print("f2 rmse: %f" % npext.rmse(f2(x*freq), sim.data[f2_p][comp_time]))

Now, let's do the same thing,
but with the same population based on context.

In [ ]:
# We could get this info in a different way, but meh!
def ctx_f(dims, comps, fns):
    fns = [radial_f(fn) for fn in fns]
    def _fn(x):
        for dim, comp, fn in zip(dims, comps, fns):
            if comp(x[dim]):
                return fn(x)
        else:
            return 0.0
    return _fn


tau = 0.025
freq = 0.5
omega = tau * 2 * np.pi * freq
with nengo.Network() as net:
    ctx_in = nengo.Node(output=None, size_in=1, label="context in")
    kick_in = nengo.Node(output=None, size_in=1, label="kick in")
    output = nengo.Node(output=None, size_in=1, label="out")
    
    ctx_dims = 1
    n_neurons = 3500 + 100 * ctx_dims
    encoders = [[np.random.uniform(-.1, .1), np.cos(theta), np.sin(theta)]
                for theta in np.random.uniform(-np.pi, (7.0 / 8) * np.pi, n_neurons)]
    oscillator = nengo.Ensemble(
            n_neurons, dimensions=2+ctx_dims, intercepts=Uniform(0.3, 1), encoders=encoders)
    nengo.Connection(ctx_in, oscillator[0])
    nengo.Connection(kick_in, oscillator, transform=[[0], [-1], [0]])
    nengo.Connection(oscillator, oscillator,
                     transform=[[0, 0, 0], [0, 1, -omega], [0, omega, 1]],
                     function=zone, synapse=tau)
    nengo.Connection(oscillator, output, function=ctx_f(
            dims=[0, 0], comps=[lambda x: x > 0.5, lambda x: x < 0.5],fns=[f1, f2]))
    
    kick = nengo.Node(lambda t: .8 if 0.1 < t < 0.2 else 0.0)
    nengo.Connection(kick, kick_in, synapse=None)
    ctx = nengo.Node(1)
    nengo.Connection(ctx, ctx_in, synapse=None)

In [ ]:
# nengo_gui.ipython.IPythonViz(net, cfg='cell2.cfg')

In [ ]:
with net:
    osc_p = nengo.Probe(oscillator, synapse=0.01)
    out_p = nengo.Probe(output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(2.2)

In [ ]:
x = np.arange(0, 1./freq, 0.001)
t = sim.trange()
plt.subplot(2, 1, 1)
plt.plot(sim.data[osc_p].T[1], sim.data[osc_p].T[2])
plt.subplot(2, 1, 2)
plt.plot(t, sim.data[out_p])
plt.plot(t, sim.data[osc_p].T[0])
plt.plot(x+0.1, f1(x*freq), c='k', lw=1)
#plt.plot(x+0.1, f2(x*freq), c='k', lw=1)

comp_time = (t > 0.1) & (t < 0.1 + 1. / freq)
print("f1 rmse: %f" % npext.rmse(f1(x*freq), sim.data[out_p][comp_time]))
#print("f2 rmse: %f" % npext.rmse(f2(x*freq), sim.data[f2_p][comp_time]))

Putting context dimensions on oscillators simply doesn't work!